# Create Your Own Data-Connected Assistant - DivaConf2024
## Gemini API: Function calling with Python

![Diva: Dive into AI](https://cdn.prod.website-files.com/6645f6ac57776cd5f3e4ac96/66633085e40eb51b599aedf3_indigo_soft_2.png "Diva: Dive into AI")

https://www.divaconf.com/

Gemini'de Fonksiyon Çağrısı, geliştiricilerin kodlarında bir fonksiyonun açıklamasını oluşturmasına ve ardından bu açıklamayı bir istekte bir dil modeline aktarmasına olanak tanır. Modelden gelen yanıt, açıklamayla eşleşen bir fonksiyonun adını ve onu çağıracak argümanları içerir.


![Function Calling Mimarisi Şeması](https://cloud.google.com/static/vertex-ai/generative-ai/docs/multimodal/images/function-calling.png "Function Calling Mimarisi")

[Resim Kaynağı](https://https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/function-calling)



### Paket Kurulumu

In [ ]:
!pip install google-generativeai==0.7.2

### Kütüphanelerin Tanımlanması ve Ayarların Yapılması

In [1]:
import google.generativeai as genai

In [2]:
# Google Cloud APIs: https://console.cloud.google.com/apis/dashboard
# Google AI Studio - API KEY: https://aistudio.google.com/app/apikey

GOOGLE_API_KEY = "{GOOGLE AI STUDIO - API KEY}"

genai.configure(api_key=GOOGLE_API_KEY)

### Örnek Fonksiyonların Tamımlanması

In [3]:
def find_movies(description: str, location: str = ""):
    """find movie titles currently playing in theaters based on any description, genre, title words, etc.

    Args:
        description: Any kind of description including category or genre, title words, attributes, etc.
        location: The city and state, e.g. San Francisco, CA or a zip code e.g. 95616
    """
    return ["Barbie", "Oppenheimer"]


def find_theaters(location: str, movie: str = ""):
    """Find theaters based on location and optionally movie title which are is currently playing in theaters.

    Args:
        location: The city and state, e.g. San Francisco, CA or a zip code e.g. 95616
        movie: Any movie title
    """
    return ["Googleplex 16", "Android Theatre"]


def get_showtimes(location: str, movie: str, theater: str, date: str):
    """
    Find the start times for movies playing in a specific theater.

    Args:
      location: The city and state, e.g. San Francisco, CA or a zip code e.g. 95616
      movie: Any movie title
      thearer: Name of the theater
      date: Date for requested showtime
    """
    return ["10:00", "11:00"]

In [4]:
functions = {
    "find_movies": find_movies,
    "find_theaters": find_theaters,
    "get_showtimes": get_showtimes,
}

model = genai.GenerativeModel(model_name="gemini-1.5-flash", tools=functions.values())

In [5]:
response = model.generate_content(
    "Mountain View'daki hangi sinemalarda Barbie filmi gösteriliyor?"
)
response.candidates[0].content.parts

[function_call {
  name: "find_theaters"
  args {
    fields {
      key: "location"
      value {
        string_value: "Mountain View, CA"
      }
    }
    fields {
      key: "movie"
      value {
        string_value: "Barbie"
      }
    }
  }
}
]

In [6]:
def call_function(function_call, functions):
    function_name = function_call.name
    function_args = function_call.args
    return functions[function_name](**function_args)


part = response.candidates[0].content.parts[0]

# Check if it's a function call; in real use you'd need to also handle text
# responses as you won't know what the model will respond with.
if part.function_call:
    result = call_function(part.function_call, functions)

print(result)

['Googleplex 16', 'Android Theatre']


In [7]:
from google.protobuf.struct_pb2 import Struct

# Put the result in a protobuf Struct
s = Struct()
s.update({"result": result})

# Update this after https://github.com/google/generative-ai-python/issues/243
function_response = genai.protos.Part(
    function_response=genai.protos.FunctionResponse(name="find_theaters", response=s)
)

# Build the message history
messages = [
    # fmt: off
    {"role": "user",
     "parts": ["Mountain View'daki hangi sinemalarda Barbie filmi gösteriliyor?."]},
    {"role": "model",
     "parts": response.candidates[0].content.parts},
    {"role": "user",
     "parts": [function_response]},
    # fmt: on
]

# Generate the next response
response = model.generate_content(messages)
print(response.text)

Mountain View'da Googleplex 16 ve Android Theatre sinemalarında Barbie filmi gösteriliyor. 



### Parallel function calls

In [8]:
def power_disco_ball(power: bool) -> bool:
    """Powers the spinning disco ball."""
    print(f"Disco ball is {'spinning!' if power else 'stopped.'}")
    return True


def start_music(energetic: bool, loud: bool, bpm: int) -> str:
    """Play some music matching the specified parameters.

    Args:
      energetic: Whether the music is energetic or not.
      loud: Whether the music is loud or not.
      bpm: The beats per minute of the music.

    Returns: The name of the song being played.
    """
    print(f"Starting music! {energetic=} {loud=}, {bpm=}")
    return "Never gonna give you up."


def dim_lights(brightness: float) -> bool:
    """Dim the lights.

    Args:
      brightness: The brightness of the lights, 0.0 is off, 1.0 is full.
    """
    print(f"Lights are now set to {brightness:.0%}")
    return True

In [9]:
# Set the model up with tools.
house_fns = [power_disco_ball, start_music, dim_lights]
# Try this out with Pro and Flash...
model = genai.GenerativeModel(model_name="gemini-1.5-flash", tools=house_fns)

# Call the API.
chat = model.start_chat()
response = chat.send_message("Burayı partiye dönüştür!")

# Print out each of the function calls requested from this single call.
for part in response.parts:
    if fn := part.function_call:
        args = ", ".join(f"{key}={val}" for key, val in fn.args.items())
        print(f"{fn.name}({args})")

power_disco_ball(power=True)
start_music(bpm=120.0, loud=True, energetic=True)
dim_lights(brightness=0.5)


In [12]:
# Simulate the responses from the specified tools.
responses = {
    "power_disco_ball": True,
    "start_music": "Diva Yorgun",
    "dim_lights": True,
}

# Build the response parts.
response_parts = [
    genai.protos.Part(function_response=genai.protos.FunctionResponse(name=fn, response={"result": val}))
    for fn, val in responses.items()
]

response = chat.send_message(response_parts)
print(response.text)

Disco topu çalışıyor, Diva Yorgun çalıyor ve ışıklar kısık. Hadi eğlenelim! 



**Kaynaklar;**


*   https://ai.google.dev/gemini-api/docs/function-calling
*   https://github.com/GoogleCloudPlatform/generative-ai/tree/main/gemini/function-calling

